Based on: https://www.tensorflow.org/text/tutorials/text_generation

Import libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

Read previous year's MA theses

In [2]:
url = tf.keras.utils.get_file('theses.txt', 'https://raw.githubusercontent.com/Joaoffg/DSS_algorithms/main/theses.txt')

# Read, then decode for py2 compat.
text = open(url, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

2351104/2334957 [==============================] - 0s 0us/step
Length of text: 2333994 characters


Convert all of the characters in the theses to numbers that are readable by an neural network (e.g. 1 = 'a'; 2 = 'b')

In [3]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

146 unique characters
tf.Tensor(
[b'\xef\xbb\xbf' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'\r'
 b'\n' b'\r' b'\n' b' ' b'\r' b'\n' b'A' b'l' b'g' b'o' b'r' b'i' b't'
 b'h' b'm' b's' b' ' b't' b'o' b'l' b'd' b' ' b'm' b'e' b' ' b't' b'o'
 b' ' b'b' b'u' b'y' b' ' b'i' b't' b'\r' b'\n' b' ' b' ' b' ' b'T' b'o'
 b' ' b'w' b'h' b'a' b't' b' ' b'e' b'x' b't' b'e' b'n' b't' b' ' b'd'
 b'o' b' ' b'A' b'm' b'a' b'z' b'o' b'n' b"'" b's' b' ' b'r' b'e' b'c'
 b'o' b'm' b'm' b'e' b'n' b'd' b'a' b't' b'i' b'o' b'n' b's' b' ' b'i'
 b'n' b'f' b'l' b'u' b'e' b'n'], shape=(101,), dtype=string)
b"\xef\xbb\xbf          \r\n\r\n \r\nAlgorithms told me to buy it\r\n   To what extent do Amazon's recommendations influen"
b'ce the choice of its customers?\r\n \r\n \r\n \r\n \r\n\r\n\r\nStudent Name:        Mihael Gelo\r\n   Student Number:'
b'    512149\r\n \r\n   Supervisor:   \t\tJo\xc3\xa3o Gon\xc3\xa7alves, MSc.\r\n \r\n   \r\n   Master Media Studies. Media & Crea'
b'tive Industries\r\nErasmus 

Create the Neural Network

In [4]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


model.summary()

(64, 100, 147) # (batch_size, sequence_length, vocab_size)
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  37632     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  150675    
                                                                 
Total params: 4,126,611
Trainable params: 4,126,611
Non-trainable params: 0
_________________________________________________________________


Train the model (you can increse the number of EPOCHS to make a better model, but it will also take longer).

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

tf.exp(mean_loss).numpy()

model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 5

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Prediction shape:  (64, 100, 147)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.990237
Epoch 1/5
361/361 [==============================] - 72s 188ms/step - loss: 2.3934
Epoch 2/5
361/361 [==============================] - 70s 189ms/step - loss: 1.3975
Epoch 3/5
361/361 [==============================] - 70s 188ms/step - loss: 1.1610
Epoch 4/5
361/361 [==============================] - 69s 188ms/step - loss: 1.0597
Epoch 5/5
361/361 [==============================] - 69s 187ms/step - loss: 0.9912


Generate text (you can specify the input to be completed by changing the sentence in prediction_input, just make sure to keep it between ' ')

In [ ]:
prediction_input = 'The research question is '

class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

start = time.time()
states = None
next_char = tf.constant([prediction_input])
result = [next_char]

for n in range(500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print("The input is: " + prediction_input)
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

The input is: The research question is 
The research question is low levels of credibility


91.1.2 Sampling
	It was expected to be satisfied with this study. This is a result, the difference between people who are highly complexely than 34% of the two Trump 2021 campaigns, mainly filled in the case of H2b, Hayes (2017) argues that some celebrity end it in group (2) belong to a study, the findings of this study is that the black - o.7, however, recommendations can provide more male models for audience. It internated decision model 4, the global individual 

________________________________________________________________________________

Run time: 2.776083469390869
